# RANS from DNS

Our goal here, in general, is to find new valid equations that describe fluid flow. 
We will try to find new closures for the steady RANS equations based on direct numerical
simulation (DNS) of a boundary layer.


## Steady RANS equations with generic Reynolds stress effects

$$
(\vec{U} \cdot \nabla) \vec{U}
+ \frac{1}{\rho} \nabla P 
- \nu \nabla^2 \vec{U}
= \mathbf{R},
$$

where in this case $\mathbf{R}$ is simply the effects of the Reynolds stresses (i.e., the opposite of the gradient), not the Reynolds stresses themselves.

Some ideas for what $\mathbf{R}$ could be:

$$
\mathbf{R} = A\nabla P^2 + B\nabla K + C \nabla \times \vec{U} 
    + D\nabla(\nabla \times \vec{U})^2
    + E \vec{U}
$$


## Algorithm

1. Pick terms (in addition to non-Reynolds stress Navier--Stokes terms).
2. Create a random list of points in space that is at least as large as the number
   of terms.
3. At each point, acquire all data for all terms for all times.
4. Average data at each point for all times.
5. Solve for coefficients using a linear model.

## Terms

$$
U \frac{\partial U}{\partial x} 
+ V \frac{\partial U}{\partial y} + W \frac{\partial U}{\partial z}
+ \frac{1}{\rho}\frac{\partial P}{\partial x} 
- \nu \left( 
    \frac{\partial^2 U}{\partial x^2}
    + \frac{\partial^2 U}{\partial y^2} 
    + \frac{\partial^2 U}{\partial z^2} 
\right)
$$

$$
= 
A \left( \frac{\partial U}{\partial x} \right)^2 
+ B \left( \frac{\partial U}{\partial y} \right)^2
+ C \left( \frac{\partial U}{\partial z} \right)^2
+ D \left( \frac{\partial P}{\partial x} \right)^2
+ E \frac{\partial^2 P}{\partial x^2}
+ F U \frac{\partial P}{\partial x}
$$

$$
U \frac{\partial V}{\partial x} 
+ V \frac{\partial V}{\partial y} 
+ W \frac{\partial V}{\partial z}
+ \frac{1}{\rho}\frac{\partial P}{\partial y} 
- \nu \left( 
    \frac{\partial^2 V}{\partial x^2}
    + \frac{\partial^2 V}{\partial y^2} 
    + \frac{\partial^2 V}{\partial z^2} 
\right)
$$

$$
= 
A \left( \frac{\partial V}{\partial x} \right)^2 
+ B \left( \frac{\partial V}{\partial y} \right)^2
+ C \left( \frac{\partial V}{\partial z} \right)^2
+ D \left( \frac{\partial P}{\partial y} \right)^2
+ E \frac{\partial^2 P}{\partial y^2}
+ F V \frac{\partial P}{\partial y}
$$

## Terms in index notation

To be general and consistent, since we don't have any x- or z-variation

$$
\frac{\partial U_i}{\partial t} + U_j \frac{\partial U_i}{\partial x_j} 
+ \frac{1}{\rho}\frac{\partial P}{\partial x_i}
- \nu \frac{\partial ^2 U_i}{\partial x_j x_j}
=
A \frac{(\partial U_i)^2}{\partial x_j \partial x_j}
+ B \frac{\partial U_j U_j}{\partial x_i}
+ C \frac{\partial P^2}{\partial x_i}
+ D \left( \frac{\partial P}{\partial x_i} \right)^2
+ E U_j \frac{\partial P}{\partial x_j}
$$

What these coefficients describe:

* $A$: The square of the velocity gradient
* $B$: The gradient of kinetic energy
* $C$: The gradient of squared pressure
* $D$: The square of the pressure gradient

Other possible quantities
* Absolute distance from a solid boundary

In [ ]:
%load_ext autoreload
%autoreload 2

import seaborn
seaborn.set_theme()

%matplotlib inline

## Using data from pyJHTDB

1. Pick a bunch of points randomly throughout the domain, at least more than the number of terms we want to test.
2. Add points in each direction for computing spatial derivatives.
3. Get $\vec{u}$, $p$, and their gradients for all time at all points in the list.
4. Calculate terms based on mean values.
5. Use a regression model to determine coefficients on each term.
6. Repeat this process to ensure the coefficients don't change?
7. Run a RANS simulation with this new model and check the results against the mean profiles.

In [ ]:
import pandas as pd

df = pd.read_hdf("data/jhtdb-transitional-bl/all-stats.h5", key="data")

In [ ]:
# Let's check continuity
div = df["dudx"] + df["dvdy"] + df["dwdz"]
div.describe()

The check above gives us an idea on how accurate these gradient calculations
are.

In [ ]:
df.columns

In [ ]:
ax = (
    df.loc[df.index.get_level_values("x")[-1000]]
    .reset_index()
    .plot(x="u", y="y", legend=False, ylabel="$U$", xlabel="$y$")
)

In [ ]:
# Check the momentum equation
from py_package import nu, rho

momx_no_res = (
    df.u * df.dudx
    + df.v * df.dudy
    + df.w * df.dudz
    + (1 / rho) * df.dpdx
    - nu * (df.d2udx2 + df.d2udy2 + df.d2udz2)
)

momx = momx_no_res + (df.duudx_fd + df.duvdy_fd)

momx.dropna().describe()

## Check the $y$-component of the momentum equation

$$
\frac{\partial V}{\partial t}
+ U \frac{\partial V}{\partial x}
+ V \frac{\partial V}{\partial y}
+ W \frac{\partial V}{\partial z}
= - \frac{1}{\rho} \frac{\partial P}{\partial y}
+ \nu \left( 
    \frac{\partial^2 V}{\partial x^2}
    + \frac{\partial^2 V}{\partial y^2}
    + \frac{\partial^2 V}{\partial z^2}
    \right)
$$

$$
- \left(
    \frac{\partial \overline{u'v'}}{\partial x}
    + \frac{\partial \overline{v'v'}}{\partial y}
    + \frac{\partial \overline{v'w'}}{\partial x}
    \right
)
$$

In [ ]:
# Check the y-component of the momentum equation
from py_package import nu, rho

momy_no_res = (
    df.u * df.dvdx
    + df.v * df.dvdy
    + df.w * df.dvdz
    + (1 / rho) * df.dpdy
    - nu * (df.d2vdx2 + df.d2vdy2 + df.d2wdz2)
)

momy = momy_no_res + (df.duvdx_fd + df.duvdy_fd)

momy_no_res.dropna().describe()

In [ ]:
momy_no_res.plot.hist()

In [ ]:
# Let's see how large the Reynold's stress residual is at different locations
df1 = df.dropna().reset_index()
df1.plot.scatter(
    x="x", y="y", color=momy_no_res.dropna().values, cmap="viridis"
)

In [ ]:
# Let's check the time-averaged profiles
import pandas as pd
import h5py
import numpy as np

def read_profiles():
    """Read profile data from JHTDB HDF5 file, and assemble into a dictionary
    of NumPy arrays.
    """
    with h5py.File(
        "data/jhtdb-transitional-bl/time-ave-profiles.h5", "r"
    ) as f:
        data = {}
        for k in f.keys():
            kn = k.split("_")[0]
            if kn.endswith("m"):
                kn = kn[:-1]
            data[kn] = f[k][()]
    # Calculate some finite difference gradients
    dx = np.gradient(data["x"])
    dy = np.reshape(np.gradient(data["y"]), (224, 1))
    # dz = np.gradient(data["z"])
    # Correct fluctuation terms according to README
    # >uum is the time-averaged of u*u (not u'*u', where u'=u-um).
    # >So time-averaged of u'*u'=uum-um*um. Same for other quantities.
    for dim in ("u", "v", "w"):
        data[f"{dim}{dim}"] = data[f"{dim}{dim}"] - data[f"{dim}"] ** 2
    # Calculate gradients
    data["dpdx"] = np.gradient(data["p"], axis=1) / dx
    data["duudx"] = np.gradient(data["uu"], axis=1) / dx
    data["duvdx"] = np.gradient(data["uv"], axis=1) / dx
    data["duvdy"] = np.gradient(data["uv"], axis=0) / dy
    data["dudx"] = np.gradient(data["u"], axis=1) / dx
    data["dudy"] = np.gradient(data["u"], axis=0) / dy
    data["d2udx2"] = np.gradient(data["dudx"], axis=1) / dx
    data["d2udy2"] = np.gradient(data["dudy"], axis=0) / dy
    data["dpdx"] = np.gradient(data["p"], axis=1) / dx
    data["dpdy"] = np.gradient(data["p"], axis=0) / dy
    # data["dwdz"] = np.gradient(data["w"], axis=1) / dz
    return data

data = read_profiles()

In [ ]:
data.keys()

In [ ]:
data["u"].shape

In [ ]:
data["x"][3000]

In [ ]:
import plotly.express as px
import plotly
import glob
from IPython.display import display, HTML

# Workaround to show LaTeX labels
plotly.offline.init_notebook_mode()
display(
    HTML(
        '<script type="text/javascript" async '
        'src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/'
        '2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
    )
)

fig = px.line(x=data["u"][:, 1500], y=data["y"], height=800).update_traces(
    line_color="green", line_dash="dash", name="DNS"
)

turbulence_model = "k-epsilon"

for ny in [15, 20, 30, 40, 60]:
    fpaths = glob.glob(
        f"sim/cases/{turbulence_model}-ny-{ny}/"
        "postProcessing/sample/*/x906.8_U.csv"
    )
    assert len(fpaths) == 1
    df2 = pd.read_csv(fpaths[0])
    fig2 = df2.plot(x="U_0", y="y", backend="plotly").update_traces(
        name=f"$N_y = {ny}$"
    )
    fig = fig.add_trace(fig2.data[0])
    fig.data[0].name = "DNS"
    fig.data[1].name = r"$k$--$\epsilon$"
fig = fig.update_layout(
    showlegend=True, xaxis=dict(title=r"$U$"), yaxis=dict(title="$y$")
)
fig.show()

In [ ]:
# TODO: Compute a bunch of quantities and add to the data
# Mean kinetic energy
# Squared gradients
# Gradients multiplied by each other
# Gradients multiplied by mean values

In [ ]:
# Compute the Reynolds stress residual as a target for an ML model
# Solve a linear regression for the coefficients of all derived terms
# Throw out terms with coefficients below a threshold

In [ ]:
# TODO: Write as a RANS model for OpenFOAM and solve this same problem there
# First run a baseline case with a high Re kOmegaSST model
# How to handle wall functions?

In [ ]:
# TODO: Check mean flow from OpenFOAM simulation matches DNS